<a href="https://colab.research.google.com/github/vvrakesh27/VENKATA-RAKESH/blob/main/DataFrame_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PySpark Installation**

In [ ]:
#install Apache Spark 3.0.1 with Hadoop 2.7 from here.
!wget https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz


# Now, we just need to unzip that folder.
!tar -xvzf spark-3.0.0-bin-hadoop2.7.tgz
!pip install findspark




import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"
import findspark
findspark.init()


**Spark Session**

In [35]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Spark Demo App").getOrCreate()
sc = spark.sparkContext


**Read Json file**

In [37]:
df = spark.read.option("multiline", "true").json("user_03.json")
df.show(truncate=0)

+-----------------------------------------+---+---------------------------+---------+-------+--------+----------------------------------------------+------+
|address                                  |age|children                   |firstName|isAlive|lastName|phoneNumbers                                  |spouse|
+-----------------------------------------+---+---------------------------+---------+-------+--------+----------------------------------------------+------+
|[New York, 10021-3100, NY, 21 2nd Street]|27 |[Catherine, Thomas, Trevor]|John     |true   |Smith   |[[212 555-1234, home], [646 555-4567, office]]|null  |
+-----------------------------------------+---+---------------------------+---------+-------+--------+----------------------------------------------+------+



**Flattening the json file**

In [45]:
from pyspark.sql.functions import explode
df_flat_ph = df.withColumn("phoneNumber",explode("phoneNumbers")).drop("phoneNumbers")
df_flat_ch = df_flat_ph.withColumn("child_name",explode("children")).drop("children")
df_flat = df_flat_ch.select("firstName","lastName","age","isAlive","spouse","child_name","address.*","phoneNumber.*")
df_flat.show(truncate=0)

+---------+--------+---+-------+------+----------+--------+----------+-----+-------------+------------+------+
|firstName|lastName|age|isAlive|spouse|child_name|city    |postalCode|state|streetAddress|number      |type  |
+---------+--------+---+-------+------+----------+--------+----------+-----+-------------+------------+------+
|John     |Smith   |27 |true   |null  |Catherine |New York|10021-3100|NY   |21 2nd Street|212 555-1234|home  |
|John     |Smith   |27 |true   |null  |Thomas    |New York|10021-3100|NY   |21 2nd Street|212 555-1234|home  |
|John     |Smith   |27 |true   |null  |Trevor    |New York|10021-3100|NY   |21 2nd Street|212 555-1234|home  |
|John     |Smith   |27 |true   |null  |Catherine |New York|10021-3100|NY   |21 2nd Street|646 555-4567|office|
|John     |Smith   |27 |true   |null  |Thomas    |New York|10021-3100|NY   |21 2nd Street|646 555-4567|office|
|John     |Smith   |27 |true   |null  |Trevor    |New York|10021-3100|NY   |21 2nd Street|646 555-4567|office|
+

**Fetch all phone Numbers whose type is Office.**

In [46]:
office_ph_nbrs_df = df_flat.select("type","number").filter(df_flat.type=="office")
office_ph_nbrs_df.show(truncate=0)

+------+------------+
|type  |number      |
+------+------------+
|office|646 555-4567|
|office|646 555-4567|
|office|646 555-4567|
+------+------------+

